## Load Packages

In [51]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
import en_core_web_sm
import pandas as pd
import numpy as np
from numpy.core.defchararray import find

In [52]:
nlp1 = en_core_web_sm.load()

C:\Users\Suzy\anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.4). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


# Labeled Data

In [53]:
#ingredients
df = pd.read_csv('tag_train.csv')  

In [54]:
df.head()

,ingredients,rand,core,category,contains,other,allergen
0,spinach leaves packed,1,spinach,spinach,NaN,NaN,NaN
1,sliced asiago cheese,1,asiago cheese,cheese,NaN,NaN,NaN
2,dried great northern beans,2,northern bean,bean,NaN,NaN,NaN
3,hamburger patties,2,hamburger,beef,NaN,NaN,NaN
4,diced tomatoes drained.,2,tomato,tomato,NaN,NaN,NaN


In [55]:
ingredients = df.ingredients.values.astype(str)
core = df.core.values.astype(str)
df['start']=find(ingredients, core)
df['end'] = df['core'].astype(str).map(len)
df['sum'] = df['start']+ df['end']
df['entity'] = 'PRODUCT'


In [56]:
subset = df[['ingredients','core','start', 'sum', 'entity']].dropna()
subset['tuples'] = [tuple(x) for x in subset[['start','sum','entity']].to_numpy()]

def formatter(x):
    return (x.ingredients, {'entities': [x.tuples]})

subset['training_set'] = subset.apply(formatter, axis=1)


## Working of NER

In [57]:
docx1 = nlp1(u"Who is Nishanth?")

In [58]:
for token in docx1.ents:
    print(token.text,token.start_char, token.end_char,token.label_)

Nishanth 7 15 DATE


In [59]:
docx2 = nlp1(u"Who is Kamal Khumar?")

In [60]:
for token in docx2.ents:
    print(token.text,token.start_char, token.end_char,token.label_)

Kamal Khumar 7 19 PERSON


## Train Data

In [61]:
TRAIN_DATA = subset.training_set.tolist()

In [62]:
TRAIN_DATA[:10]

[('spinach leaves packed', {'entities': [(0, 7, 'PRODUCT')]}),
 ('sliced asiago cheese', {'entities': [(7, 20, 'PRODUCT')]}),
 ('dried great northern beans', {'entities': [(12, 25, 'PRODUCT')]}),
 ('hamburger patties', {'entities': [(0, 9, 'PRODUCT')]}),
 ('diced tomatoes drained.', {'entities': [(6, 12, 'PRODUCT')]}),
 ('head napa cabbage chopped', {'entities': [(5, 17, 'PRODUCT')]}),
 ('uncooked cellentani (corkscrew) pasta',
  {'entities': [(32, 37, 'PRODUCT')]}),
 ('dry unseasoned bread cubes', {'entities': [(15, 20, 'PRODUCT')]}),
 ('spelt flour', {'entities': [(6, 11, 'PRODUCT')]}),
 ('dry curd cottage cheese', {'entities': [(9, 23, 'PRODUCT')]})]

## Define our variables

In [63]:
model = None
output_dir=Path("./NERmodel")
n_iter=10

## Load the model

In [64]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


## Set up the pipeline

In [65]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

## Train the Recognizer

In [66]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(
C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:635: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'en'.
  proc.begin_training(
  0%|▏                                                                                | 3/1409 [00:00<03:05,  7.58it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "prepared 16-inch pizza dough pastry" with entities "[(-1, 10, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignm

  4%|██▊                                                                             | 50/1409 [00:03<01:38, 13.79it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "cremini mushrooms chopped" with entities "[(-1, 18, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
  4%|██▉                                                                             | 52/1409 [00:04<01:52, 12.03it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "avocados halved pitted flesh spooned out" with entities "[(0, 7, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignore

  8%|██████▏                                                                        | 111/1409 [00:09<01:38, 13.24it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "seasoned diced tomatoes" with entities "[(15, 21, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "chocolate-flavored syrup" with entities "[(-1, 14, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
  8%|██████▋                                                                        

 11%|█████████                                                                      | 161/1409 [00:13<01:34, 13.15it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "ripe avocados cubed" with entities "[(5, 12, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
 12%|█████████▏                                                                     | 163/1409 [00:13<01:31, 13.69it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "chopped roasted peanuts" with entities "[(16, 22, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  g

 15%|███████████▋                                                                   | 209/1409 [00:16<01:34, 12.69it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "garbanzo beans half the liquid reserved" with entities "[(0, 13, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
 15%|████████████                                                                   | 215/1409 [00:17<01:26, 13.81it/s]C:\Users\Suzy\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "fresh green chilies for garnish" with entities "[(-1, 12, 'PRODUCT')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be 

...

{'ner': 490.4412673411107}


  0%|                                                                                         | 0/1409 [00:00<?, ?it/s]

{'ner': 527.8280798870642}


100%|██████████████████████████████████████████████████████████████████████████████| 1409/1409 [03:08<00:00,  7.46it/s]

{'ner': 532.3521079112674}


## Apply model on recipelist

In [67]:
#testdata
recipelist = np.loadtxt('newrecipeinputlist.txt',delimiter = '\n', dtype=str)
len(recipelist)

35399

In [68]:
# for text, _ in TRAIN_DATA[:30]:
#     doc = nlp(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [69]:
def extract_core(text):
    ingredient = nlp2(text)
    for ent in ingredient.ents:
        return ent.text 

In [70]:
# import ast
# df1 = pd.read_csv('recipe_ingredients_wenqi.csv') 
# df1['ingredientlist'] = df1.ingredients2.apply(lambda x: ast.literal_eval(x))
# df2 = df1[['recipe_id', 'recipe_name', 'ingredientlist']]
# df2.head()

In [71]:
# df3 = df2.explode('ingredientlist')
# df3['ingredientlist_lower'] = df3['ingredientlist'].str.lower()

In [72]:
# ingredient_unique = list(set(df3.ingredientlist_lower))

In [73]:
# ingredient_unique[0:10]

In [74]:
# ingredient_unique['core'] = ingredient_unique['ingredients'].apply(lambda x: extract_core(x))

# Save Model

In [75]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to NERmodel


# Test Model



In [76]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA[:30]:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Loading from NERmodel
Entities [('apples', 'PRODUCT')]
Tokens [('apples', 'PRODUCT', 3), ('cored', '', 2), ('and', '', 2), ('sliced', '', 2)]
Entities [('ham', 'PRODUCT')]
Tokens [('bone', '', 2), ('-', '', 2), ('in', '', 2), ('fully', '', 2), ('cooked', '', 2), ('ham', 'PRODUCT', 3), ('steak', '', 2)]
Entities [('mushroom', 'PRODUCT')]
Tokens [('thinly', '', 2), ('sliced', '', 2), ('mushroom', 'PRODUCT', 3)]
Entities [('scallops', 'PRODUCT')]
Tokens [('large', '', 2), ('scallops', 'PRODUCT', 3)]
Entities [('hot sauce', 'PRODUCT')]
Tokens [('chipotle', '', 2), ('-', '', 2), ('flavored', '', 2), ('hot', 'PRODUCT', 3), ('sauce', 'PRODUCT', 1)]
Entities [('vegetarian fish', 'PRODUCT')]
Tokens [('vegetarian', 'PRODUCT', 3), ('fish', 'PRODUCT', 1), ('sauce', '', 2)]
Entities [('green beans', 'PRODUCT')]
Tokens [('chopped', '', 2), ('green', 'PRODUCT', 3), ('beans', 'PRODUCT', 1)]
Entities [('vermouth', 'PRODUCT')]
Tokens [('fluid', '', 2), ('ounce', '', 2), ('dry', '', 2), ('vermouth', 'PRO

In [77]:
recipeingredientlist = recipelist.tolist()

In [78]:
for text in recipeingredientlist[:30]:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('apple cider', 'PRODUCT')]
Tokens [('apple', 'PRODUCT', 3), ('cider', 'PRODUCT', 1), ('divided', '', 2)]
Entities [('brown', 'PRODUCT')]
Tokens [('brown', 'PRODUCT', 3), ('sugar', '', 2)]
Entities [('caraway', 'PRODUCT')]
Tokens [('caraway', 'PRODUCT', 3), ('seeds', '', 2)]
Entities [('apples', 'PRODUCT')]
Tokens [('granny', '', 2), ('smith', '', 2), ('apples', 'PRODUCT', 3), ('sliced', '', 2)]
Entities [('pepper', 'PRODUCT')]
Tokens [('ground', '', 2), ('black', '', 2), ('pepper', 'PRODUCT', 3)]
Entities []
Tokens [('ingredientlist_lower', '', 2)]
Entities []
Tokens [('kosher', '', 2), ('salt', '', 2)]
Entities [('onion', 'PRODUCT')]
Tokens [('large', '', 2), ('onion', 'PRODUCT', 3)]
Entities [('pork belly', 'PRODUCT')]
Tokens [('pork', 'PRODUCT', 3), ('belly', 'PRODUCT', 1)]
Entities []
Tokens [('rub', '', 2)]
Entities [('sauerkraut', 'PRODUCT')]
Tokens [('sauerkraut', 'PRODUCT', 3), ('drained', '', 2)]
Entities [('paprika', 'PRODUCT')]
Tokens [('smoked', '', 2), ('paprika

In [79]:
recipe_core = []
#function applying through list
for text in recipeingredientlist:
    ingredient = nlp2(text)
    recipe_core.extend([(ent.text) for ent in ingredient.ents])


In [80]:
#per ingredient line
def extract_core(text):
    ingredient = nlp2(text)
    for ent in ingredient.ents:
        return ent.text 

In [81]:
extract_core('pulled pork')

'pork'

In [82]:
recipe_df = pd.DataFrame(recipeingredientlist, columns = ['ingredient'])
recipe_df['core'] = recipe_df['ingredient'].apply(lambda x: extract_core(x))
recipe_df.head(20)

,ingredient,core
0,apple cider divided,apple cider
1,brown sugar,brown
2,caraway seeds,caraway
3,granny smith apples sliced,apples
4,ground black pepper,pepper
5,ingredientlist_lower,None
6,kosher salt,None
7,large onion,onion
8,pork belly,pork belly
9,rub,None


In [83]:
recipe_df.to_csv('recipe_core.csv',index=False)

In [84]:
# import ast
# df1 = pd.read_csv('recipe_ingredients_wenqi.csv') 
# df1['ingredientlist'] = df1.ingredients2.apply(lambda x: ast.literal_eval(x))
# df2 = df1[['recipe_id', 'recipe_name', 'ingredientlist']]
# df2.head()

In [85]:
# df3 = df2.explode('ingredientlist')
# df3['ingredientlist_lower'] = df3['ingredientlist'].str.lower()

In [86]:
# df3['core'] = df3['ingredientlist_lower'].apply(lambda x: extract_core(x))

# Evaluation

In [90]:
#test set
df_test = pd.read_csv('real_tag.csv') 
true_labels = df_test.core

In [89]:
df_test.head()

,ingredients,core
0,gallons water divided,water
1,ground turkey breast patties,turkey breast
2,head savoy cabbage quartered and cored,cabbage
3,shredded baby carrots,carrots
4,cinnamon stick,cinnamon


In [92]:
predict_labels = df_test['ingredients'].apply(lambda x: extract_core(x))
predict_labels[0:15]

0              water
1      turkey breast
2            cabbage
3            carrots
4               None
5           stuffing
6               None
7     ranch dressing
8       wine vinegar
9               None
10              None
11      curry powder
12           halibut
13             metal
14            orange
Name: ingredients, dtype: object

In [97]:
ingredients = df_test.ingredients.values.astype(str)
core = df_test.core.values.astype(str)
df_test['start']=find(ingredients, core)
df_test['end'] = df_test['core'].astype(str).map(len)
df_test['sum'] = df_test['start']+ df_test['end']
df_test['entity'] = 'PRODUCT'

In [99]:
subset_test = df_test[['ingredients','core','start', 'sum', 'entity']]
subset_test['tuples'] = [tuple(x) for x in subset_test[['start','sum','entity']].to_numpy()]

def formatter(x):
    return (x.ingredients, {'entities': [x.tuples]})

subset_test['test_set'] = subset_test.apply(formatter, axis=1)
examples = subset_test.test_set.tolist()

In [103]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

# example run

ner_model = spacy.load("./NERmodel") # for spaCy's pretrained use 'en_core_web_sm'
results = evaluate(ner_model, examples)

IndexError: list index out of range

In [104]:
# def evaluate(ner_model, examples):
#     scorer = Scorer()
#     for input_, annot in examples:
#         doc_gold_text = ner_model.make_doc(input_)
#         gold = GoldParse(doc_gold_text, entities=annot)
#         pred_value = ner_model(input_)
#         scorer.score(pred_value, gold)
#     return scorer.scores
scorer = Scorer()
for i in df_test.ingredients:
    scorer.score(predict_labels, true_labels)
results = scorer.scores

# ner_model = spacy.load("./NERmodel") # for spaCy's pretrained use 'en_core_web_sm'
# results = evaluate(ner_model, examples)

AttributeError: 'Series' object has no attribute 'orig_annot'